# Speech recognition for isolated words

# Mathematical background

## Feature Extraction
Feature extraction is the method of transforming the input data into a set of features which represents as closely as possible the input data. Audio feature extraction use Digital Signal Processing techniques to convert the audio signals into numerical feature sets which can then be used for training our machine learning models.

## MFCC

MFCC stands for mel-frequency cepstral coefficients. They were introduced by Davis and Mermelstein in the 1980's. These are the most commonly used features in automatic speech recognition. MFCCs are derived from a type of cepstral representation of the audio clip. A cepstrum is the result of taking the inverse fourier transform of the logarithm of the estimated spectrum of the signal. The power cepstrum in particular finds applications in the analysis of human speech. The main difference between the MFC and cepstrum is that in MFC, equal spacing of the frequency bands takes place which approximates the human auditory system’s response more closely. 

## Import library

In [1]:
import doctest
import struct
import wave
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from numpy import fft

## Useful functions

In [25]:
def readwav(fname):
    """
    Parameters
    ----------
    fname : str
        The filename of the wave file to process

    Returns
    -------
    
    (x, fs) : tuple
    
    x : np.array
        The 1-dim. PCM audio signal

    fs : int
        The sampling frequency
    """
    with wave.open(fname) as wav:
        n = wav.getnframes()
        frames = wav.readframes(n)
        x = struct.unpack("h" * n, frames)
        fs = wav.getframerate()
    return np.array(x), fs

Test the readwav fonction

In [26]:
test=readwav("digits/train/ae_1a.wav")
fs=test[-1]
print(len(test[0]))
print(fs*0.001)

6964
8.0


Mel Scale: Corresponds to an approximation of the psychological sensation of pitch of a sound.

Hertz to mel conversion is computed according to:

In [69]:
def mel(f):
    """
    Frequency converter from Hz to Mel

    Parameters
    ----------
    f : int, float or np.array
        The frequency (Hz)

    Returns
    -------
    mel : float or np.array
        The frequency (Mel)

    Examples
    --------
    >>> x = np.linspace(200, 4000, 20)
    >>> with np.printoptions(precision=2):
    ...     print(mel(x))
    [  283.23  509.38  697.65  858.93  999.99 1125.34 ... 2146.06]
    """
 
    return 2595*np.log10(1+f/700)


def hertz(m):
    """
    Frequency converter from Mel to Hz

    Parameters
    ----------
    mel : int, float or np.array
        The frequency (Mel)

    Returns
    -------
    f : float or np.array
        The frequency (Hz)

    Examples
    --------
    >>> x = np.linspace(200, 2000, 20)
    >>> with np.printoptions(precision=2):
    ...     print(hertz(x))
    [  135.93  209.24  288.97  375.7   470.04  572.64 ... 3428.68]
    """
    return 700 * (10**(m/2595)-1)

Test the convertion

In [70]:
x1 = np.linspace(200, 4000, 20)
x2 = np.linspace(200, 2000, 20)
print('Frequency converter from Hz to Mel \n')
with np.printoptions(precision=2):
        print(mel(x1))
print('\n')
print('Frequency converter from Mel to Hz \n')
with np.printoptions(precision=2):
        print(hertz(x2))        

Frequency converter from Hz to Mel 

[ 283.23  509.38  697.65  858.93  999.99 1125.34 1238.13 1340.65 1434.62
 1521.36 1601.89 1677.05 1747.51 1813.83 1876.45 1935.78 1992.14 2045.82
 2097.06 2146.06]


Frequency converter from Mel to Hz 

[ 135.93  209.24  288.97  375.7   470.04  572.64  684.25  805.64  937.68
 1081.3  1237.51 1407.42 1592.23 1793.25 2011.9  2249.72 2508.4  2789.76
 3095.8  3428.68]


In [68]:
def melbank(nfilt, win_n, fs):
    """
    Computing of the MFCC coefficients of a whole wave file

    Parameters
    ----------
    nfilt : int
        The number of melbank filters

    win_n : int
        The window size (in samples)

    fs : int
        The sampling frequency (Hz)

    Returns
    -------
    melb : np.array
        The 2-dim. (nfilt, win_n/2+1) mel-bank filters

    Examples
    --------
    >>> melb = melbank(nfilt=24, win_n=256, fs=8000)
    >>> melb.shape
    (24, 129)
    >>> with np.printoptions(precision=2, suppress=True, edgeitems=6):
    ...     print(melb)
    [[0.   1.   0.5  0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.5  1.   0.5  0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.5  1.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     ...
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.36 0.27 0.18 0.09 0.   0.  ]]
    >>> with np.printoptions(precision=2, suppress=True, edgeitems=6):
    ...     print(melb[-1])
    [0.   0. ... 0.1  0.2 ... 1.   0.91 0.82 0.73 ... 0.18 0.09 0.   0.  ]
    """
    melb = np.zeros((nfilt, win_n // 2 + 1))
    melstep = mel(fs / 2) / (nfilt + 1)

    for i in range(nfilt):
        left = int(hertz(melstep * i) * win_n / fs)
        middle = int(hertz(melstep * (i + 1)) * win_n / fs)
        right = int(hertz(melstep * (i + 2)) * win_n / fs)

        melb[i, left : middle + 1] = (
            np.arange(middle - left + 1) * 1.0 / (middle - left)
        )
        melb[i, middle : right + 1] = 1.0 - np.arange(
            right - middle + 1
        ) * 1.0 / (right - middle)

    return melb

In [73]:
melb = melbank(nfilt=24, win_n=256, fs=8000)
melb.shape

(24, 129)

In [74]:
with np.printoptions(precision=2, suppress=True, edgeitems=6):
    print(melb)

[[0.   1.   0.5  0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.5  1.   0.5  0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.5  1.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 ...
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   ... 0.36 0.27 0.18 0.09 0.   0.  ]]


In [6]:
def mfcc(y, melb, nc):
    """
    Computing of the MFCC coefficients of a single frame

    Parameters
    ----------
    y : np.array
        The 1-dim. frame signal

    melb: np.array
        The 2-dim. mel-bank filters

    nc: int
        The number of coefficients

    Returns
    -------
    cc : np.array
        The 1-dim cepstral coefficients

    Examples
    --------
    >>> int16 = np.iinfo(np.int16)
    >>> rng = np.random.default_rng(0)
    >>> y = rng.integers(int16.min, int16.max, 256)
    >>> melb = melbank(nfilt=24, win_n=256, fs=8000)
    >>> cc = mfcc(y, melb, nc=13)
    >>> cc.shape
    (13,)
    """
    pass

In [7]:
def wav2mfcc(fname, nc=13, nfilt=24, win=0.032, step=0.010):
    """
    Computing of the MFCC coefficients of a whole wave file
    
    Parameters
    ----------
    fname : str
        The filename of the wave file to process
    
    nc : int
        The number of cepstral coefficients
        
    nfilt : int
        The number of melbank filters
    
    win : int
        The window size (s)
    
    step : int
        The step size (s)

    Returns
    -------
    c : np.array
        The 2-dim cepstral coefficients of the whole file
    """
    pass

In [8]:
def euclid(v1, v2):
    """    
    Computation of the Euclidean distance between the MFCC frames
        
    Parameters
    ----------
    v1 : np.array
        1-dim. MFCC frame
    
    v2 : np.array
        1-dim. MFCC frame

    Returns
    -------
    dist : int
        The Euclidean distance value
    """
    pass

In [9]:
def preemph(x, alpha=0.97):
    """
    (Optional)
    
    Applying the pre-emphasis step to the PCM audio signal:
    to decrease high frequency energy
        
    Parameters
    ----------
    x : np.array
        The 1-dim. PCM audio signal
    
    alpha : int 
        The pre-emphasis coefficient

    Returns
    -------
    x_pre : np.array
        The 1-dim. filtered audio signal
    """
    pass

In [10]:
def dtw(c1, c2):
    """    
    Computation of DTW between MFCC signals
        
    Parameters
    ----------
    c1 : np.array
        2-dim. MFCC signal
    
    c1 : np.array
        2-dim. MFCC signal

    Returns
    -------
    dist : int
        The DTW distance value
    """
    pass

The computation script

In [11]:
pass

---